# Week 38 
In this section, we implement three binary classifiers to predict wheher a question is answerable by the data set or not. 

The first model is a Logistic Regression (GloVe), where we use the gensim library by translating each data set to english.

The second is a Logistic Regression with the BPEmb tokenizer; that is, we take the words from the langaueg and put them into smaller parts, and learn embeddings from these parts. We 

The third is a fine-tuning of the pre-trained bert-base-multilingual-cased model.

Results and further explanations can be found in the report; overall, BERT outperforms.

In [1]:
%pip install datasets
%pip install googletrans==3.1.0a0
%pip install spicy
%pip install accelerate -U
%pip install transformers[torch]
%pip install bpemb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
zsh:1: no matches found: transformers[torch]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#%run dict_maker_lang.py generate dictionaries (they are in the .pkl files, this one takes 3 hours to run)

In [3]:
import pandas as pd
import copy
import numpy as np
import torch
import nltk
import string
nltk.download('punkt')
import pickle
from bpemb import BPEmb
# stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')

from datasets import load_dataset, DatasetDict
from transformers import (
    AutoModelForSequenceClassification,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    AutoConfig,
    DataCollatorWithPadding,
    AdamW,
    get_linear_schedule_with_warmup
)
from torch.utils.data import DataLoader
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import LambdaLR
from datasets import DatasetDict

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import gensim.downloader
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
import random
from scipy.sparse import hstack as sparse_hstack

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kaspe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kaspe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
%run dict_maker_lang.py 

In [15]:
# If the .pkl files wont load, you need to run the dict_maker_lang.py file in the cell above. This file generates the translated dictionary.

file_path = 'train_set_dict.pkl'
with open(file_path, 'rb') as file:
    loaded_data = pickle.load(file)
train_set_dict = loaded_data['any']
print(train_set_dict)

file_path = 'val_set_dict.pkl'
with open(file_path, 'rb') as file:
    loaded_data = pickle.load(file)
val_set_dict = loaded_data['any']
print(val_set_dict)

{'indonesian': Dataset({
    features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url', 'question_words', 'answer_words', 'doc_words', 'doc_text_words', 'question_text_eng', 'doc_plaintext_eng', 'question_words_eng', 'doc_text_words_eng'],
    num_rows: 11394
}), 'bengali': Dataset({
    features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url', 'question_words', 'answer_words', 'doc_words', 'doc_text_words', 'question_text_eng', 'doc_plaintext_eng', 'question_words_eng', 'doc_text_words_eng'],
    num_rows: 4779
}), 'arabic': Dataset({
    features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url', 'question_words', 'answer_words', 'doc_words', 'doc_text_words', 'question_text_eng', 'doc_plaintext_eng', 'question_words_eng', 'doc_text_words_eng'],
    num_rows: 29598
})}


In [16]:
file_path = 'val_set_dict.pkl'

with open(file_path, 'rb') as file:
    loaded_data = pickle.load(file)

val_set_dict = loaded_data['any']

print(val_set_dict)

{'indonesian': Dataset({
    features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url', 'question_words', 'answer_words', 'doc_words', 'doc_text_words', 'question_text_eng', 'doc_plaintext_eng', 'question_words_eng', 'doc_text_words_eng'],
    num_rows: 1191
}), 'bengali': Dataset({
    features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url', 'question_words', 'answer_words', 'doc_words', 'doc_text_words', 'question_text_eng', 'doc_plaintext_eng', 'question_words_eng', 'doc_text_words_eng'],
    num_rows: 224
}), 'arabic': Dataset({
    features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url', 'question_words', 'answer_words', 'doc_words', 'doc_text_words', 'question_text_eng', 'doc_plaintext_eng', 'question_words_eng', 'doc_text_words_eng'],
    num_rows: 1902
})}


In [19]:
languages = ['indonesian', 'bengali', 'arabic']

### Logistic Regression

In [6]:
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [8]:
nlp = spacy.load('en_core_web_sm')
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [9]:
def run_classifier(X_train, y_train, X_test, y_test):
  # Define hyperparams for search
  # C = np.logspace(-6, 2, 50)
  # warm_start = [False, True]
  # class_weight = ['balanced', None]

  # hp = {"C": C, "warm_start": warm_start, 'class_weight': class_weight}

  classifier = LogisticRegression(penalty='l2', max_iter=1000)
  # classifier_random = RandomizedSearchCV(
  #     estimator=classifier,
  #     param_distributions=hp,
  #     n_iter=100,
  #     cv=5,
  #     verbose=2,
  #     random_state=1000,
  #     n_jobs=-1,
  #     scoring='f1'
  # )

  # classifier_random.fit(X_train, y_train)
  # print(classifier_random.best_params_)
  # print(classifier_random.best_score_)
  # model = classifier_random.best_estimator_
  classifier.fit(X_train,y_train)
  preds = classifier.predict(X_test)
  print(classification_report(y_test, preds))

def get_unigram_features(dataset, vectorizer):
  X1 = vectorizer.transform(dataset[:,0])
  X2 = vectorizer.transform(dataset[:,1])
  X = sparse_hstack([X1, X2], format='csr')
  y = list(dataset[:,2])
  return X,y

In [20]:
for lang in languages:
  print('language: ', lang)
  y_train = [0 if not answer else 1 for answer in train_set_dict[lang]['answer_words']]
  y_val = [0 if not answer else 1 for answer in val_set_dict[lang]['answer_words']]

  #yy = [int(value) for value in yy]
  train_logistic = np.column_stack((train_set_dict[lang]['question_text_eng'], train_set_dict[lang]['doc_plaintext_eng'], y_train))

  val_logistic = np.column_stack(([val_set_dict[lang]['question_text_eng'], val_set_dict[lang]['doc_plaintext_eng'], y_val]))
  # Run the regression
  vectorizer = CountVectorizer()
  vectorizer.fit(train_logistic[:,0])
  X_train,y_train = get_unigram_features(train_logistic, vectorizer)
  X_val,y_val = get_unigram_features(val_logistic, vectorizer)
  run_classifier(X_train, y_train, X_val, y_val)

language:  indonesian
              precision    recall  f1-score   support

           0       0.68      0.74      0.71       597
           1       0.71      0.64      0.67       594

    accuracy                           0.69      1191
   macro avg       0.69      0.69      0.69      1191
weighted avg       0.69      0.69      0.69      1191

language:  bengali
              precision    recall  f1-score   support

           0       0.63      0.70      0.66       112
           1       0.66      0.60      0.63       112

    accuracy                           0.65       224
   macro avg       0.65      0.65      0.65       224
weighted avg       0.65      0.65      0.65       224

language:  arabic
              precision    recall  f1-score   support

           0       0.75      0.79      0.77       956
           1       0.77      0.73      0.75       946

    accuracy                           0.76      1902
   macro avg       0.76      0.76      0.76      1902
weighted avg   

c:\Users\kaspe\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Good results for Logistic Regression shows that the data is linear seperable

# Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
for lang in languages:
  print('language: ', lang)
  y_train = [0 if not answer else 1 for answer in train_set_dict[lang]['answer_words']]
  y_val = [0 if not answer else 1 for answer in val_set_dict[lang]['answer_words']]
  train_logistic = np.column_stack((train_set_dict[lang]['question_text_eng'], train_set_dict[lang]['doc_plaintext_eng'], y_train))
  val_logistic = np.column_stack(([val_set_dict[lang]['question_text_eng'], val_set_dict[lang]['doc_plaintext_eng'], y_val]))
  # Run the regression
  vectorizer = CountVectorizer()
  vectorizer.fit(train_logistic[:,0])
  X_train,y_train = get_unigram_features(train_logistic, vectorizer)
  X_val,y_val = get_unigram_features(val_logistic, vectorizer)

  rf_classifier = RandomForestClassifier(n_estimators=100, random_state=0)
  rf_classifier.fit(X_train, y_train)
  y_val_pred = rf_classifier.predict(X_val)
  accuracy = accuracy_score(y_val, y_val_pred)
  print(f"Accuracy on the validation set: {accuracy}")


language:  indonesian
Accuracy on the validation set: 0.7397145256087322
language:  bengali
Accuracy on the validation set: 0.7455357142857143
language:  arabic
Accuracy on the validation set: 0.7760252365930599


# Logistic Regression with Bpemb

In [23]:
bpemb_id = BPEmb(lang='id', dim=100, vs=25000)
bpemb_bn = BPEmb(lang='bn', dim=100, vs=25000)
bpemb_ar = BPEmb(lang='ar', dim=100, vs=25000)

bpemp_mapping = {
    "indonesian": "id",
    "bengali": "bn",
    "arabic": "ar"
  }


def get_bpemb_features(dataset, bpemb):
  X = [bpemb.embed(x).mean(0) for x in tqdm(dataset[:,0])]
  y = list(dataset[:,1])
  return X,y


downloading https://nlp.h-its.org/bpemb/id/id.wiki.bpe.vs25000.model


100%|██████████| 650018/650018 [00:00<00:00, 6720534.75B/s]


downloading https://nlp.h-its.org/bpemb/id/id.wiki.bpe.vs25000.d100.w2v.bin.tar.gz


100%|██████████| 9465922/9465922 [00:00<00:00, 11037977.69B/s]


downloading https://nlp.h-its.org/bpemb/bn/bn.wiki.bpe.vs25000.model


...

100%|██████████| 9517491/9517491 [00:00<00:00, 10056916.51B/s]


downloading https://nlp.h-its.org/bpemb/ar/ar.wiki.bpe.vs25000.model


100%|██████████| 742254/742254 [00:00<00:00, 4917327.82B/s]


downloading https://nlp.h-its.org/bpemb/ar/ar.wiki.bpe.vs25000.d100.w2v.bin.tar.gz


100%|██████████| 9491724/9491724 [00:00<00:00, 10723661.73B/s]


In [25]:
for lang in languages:
    print('language: ', lang)
    if lang in bpemp_mapping:
        bpemb_model = globals()[f'bpemb_{bpemp_mapping[lang]}']  # Get the corresponding BPEmb model
    else:
        print(f"No BPEmb model found for language: {lang}")
        continue

    y_train = [0 if not answer else 1 for answer in train_set_dict[lang]['answer_words']]
    y_val = [0 if not answer else 1 for answer in val_set_dict[lang]['answer_words']]

    X_train = [item['question_text'] + ' [SEP] ' + item['document_plaintext'] for item in train_set_dict[lang]]
    X_val = [item['question_text'] + ' [SEP] ' + item['document_plaintext'] for item in val_set_dict[lang]]

    X_train, y_train = get_bpemb_features(np.transpose((X_train, y_train)), bpemb_model)
    X_val, y_val = get_bpemb_features(np.transpose((X_val, y_val)), bpemb_model)
    run_classifier(X_train, y_train, X_val, y_val)


language:  indonesian


100%|██████████| 1191/1191 [00:00<00:00, 3484.74it/s]


              precision    recall  f1-score   support

           0       0.71      0.72      0.72       597
           1       0.72      0.70      0.71       594

    accuracy                           0.71      1191
   macro avg       0.71      0.71      0.71      1191
weighted avg       0.71      0.71      0.71      1191

language:  bengali


100%|██████████| 224/224 [00:00<00:00, 2787.72it/s]


              precision    recall  f1-score   support

           0       0.73      0.63      0.68       112
           1       0.68      0.77      0.72       112

    accuracy                           0.70       224
   macro avg       0.70      0.70      0.70       224
weighted avg       0.70      0.70      0.70       224

language:  arabic


100%|██████████| 1902/1902 [00:00<00:00, 3150.92it/s]


              precision    recall  f1-score   support

           0       0.73      0.75      0.74       956
           1       0.74      0.73      0.73       946

    accuracy                           0.74      1902
   macro avg       0.74      0.74      0.74      1902
weighted avg       0.74      0.74      0.74      1902



# Binary Classification, Transformer model
The code below was inspired by https://huggingface.co/docs/transformers/tasks/sequence_classification

If you're using google colab, run the cell below and restart runtime

In [ ]:
%%capture
!pip install transformers datasets evaluate
!pip install transformers[torch]
!pip install datasets
!pip install sklearn
!pip install bertviz

In [ ]:
def enforce_reproducibility(seed=42):
    # Sets seed manually for both CPU and CUDA
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # For atomic operations there is currently
    # no simple way to enforce determinism, as
    # the order of parallel operations is not known.
    # CUDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # System based
    random.seed(seed)
    np.random.seed(seed)

In [ ]:
dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"]
validation_set = dataset["validation"]

languages = ['indonesian', 'bengali', 'arabic']

train_set_dict = {}
val_set_dict = {}

for language in languages:
    print("Adding ", language, "to dict")
    train_set_dict[language] = train_set.filter(lambda example: example["language"] == language)
    val_set_dict[language] = validation_set.filter(lambda example: example["language"] == language)

Adding  indonesian to dict
Adding  bengali to dict
Adding  arabic to dict


In [ ]:
# Making sure that our dataset has the same shape as SquAD_V2

def reformat_example(example, idx):
    #example["id"] = str(idx)

    # Rename columns
    example.pop("document_title")
    example["text"] = example.pop("question_text") + example.pop("document_plaintext")
    #example["question"] =

    example['label'] = int(len(example['annotations']['answer_text'][0]) > 0)

    # Reformat the answers structure
    example.pop("annotations")

    # example["answers"] = {
    #     "text": annotations["answer_text"],
    #     "answer_start": annotations["answer_start"]
    # }




    return example

TEST_SIZE = 0.15

def training_split(train_dict, val_dict):
    dict_list = DatasetDict()

    for key in train_dict.keys():
        hugging_dict = DatasetDict()

        # Transform the train dataset
        hugging_dict['train'] = train_dict[key].select_columns(['question_text', 'document_title', 'document_plaintext', 'annotations'])
        hugging_dict['train'] = hugging_dict['train'].map(reformat_example, with_indices=True)

        # Transform the validation dataset
        hugging_dict['validation'] = val_dict[key].select_columns(['question_text', 'document_title', 'document_plaintext', 'annotations'])
        hugging_dict['validation'] = hugging_dict['validation'].map(reformat_example, with_indices=True)

        dict_list[key] = hugging_dict

    return dict_list


In [ ]:
tydiqa = training_split(train_set_dict, val_set_dict)

In [ ]:
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "no_amswer", 1: "Answer"}
label2id = {"no_answer": 0, "Answer": 1}

## Indonesian

In [ ]:
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")

enforce_reproducibility()

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
tokenized_tydiqa_indo = tydiqa['indonesian'].map(preprocess_function, batched=True)
tokenized_tydiqa_ber = tydiqa['bengali'].map(preprocess_function, batched=True)
tokenized_tydiqa = tydiqa['arabic'].map(preprocess_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="indonesian",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps = 100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_tydiqa_indo["train"],
    eval_dataset=tokenized_tydiqa_indo["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("indonesian_classifier")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.367200,0.304980,0.877414
2,0.216800,0.315467,0.887490


In [ ]:
import os
from google.colab import files

# Zip the model directory
!zip -r indonesian_classifier.zip indonesian_classifier

  adding: indonesian_classifier/ (stored 0%)
  adding: indonesian_classifier/tokenizer.json (deflated 67%)
  adding: indonesian_classifier/special_tokens_map.json (deflated 42%)
  adding: indonesian_classifier/tokenizer_config.json (deflated 76%)
  adding: indonesian_classifier/vocab.txt (deflated 45%)
  adding: indonesian_classifier/pytorch_model.bin (deflated 7%)
  adding: indonesian_classifier/training_args.bin (deflated 50%)
  adding: indonesian_classifier/config.json (deflated 54%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Bengali

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenized_tydiqa_ber = tydiqa['bengali'].map(preprocess_function, batched=True)

Map:   0%|          | 0/4779 [00:00<?, ? examples/s]

Map:   0%|          | 0/224 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="bengali_classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps = 100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_tydiqa["train"],
    eval_dataset=tokenized_tydiqa["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("bengali_classifier")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.475400,0.447014,0.799107
2,0.290800,0.439516,0.825893


In [ ]:
BENGALI

In [ ]:
import os
from google.colab import files

# Zip the model directory
!zip -r bengali_classifier.zip bengali_classifier

  adding: bengali_classifier/ (stored 0%)
  adding: bengali_classifier/runs/ (stored 0%)
  adding: bengali_classifier/runs/Nov01_08-28-23_e5a9b7b4a2c7/ (stored 0%)
  adding: bengali_classifier/runs/Nov01_08-28-23_e5a9b7b4a2c7/events.out.tfevents.1698827305.e5a9b7b4a2c7.12314.0 (deflated 58%)
  adding: bengali_classifier/checkpoint-299/ (stored 0%)
  adding: bengali_classifier/checkpoint-299/rng_state.pth (deflated 25%)
  adding: bengali_classifier/checkpoint-299/scheduler.pt (deflated 56%)
  adding: bengali_classifier/checkpoint-299/tokenizer.json (deflated 67%)
  adding: bengali_classifier/checkpoint-299/special_tokens_map.json (deflated 42%)
  adding: bengali_classifier/checkpoint-299/tokenizer_config.json (deflated 76%)
  adding: bengali_classifier/checkpoint-299/vocab.txt (deflated 45%)
  adding: bengali_classifier/checkpoint-299/optimizer.pt (deflated 53%)
  adding: bengali_classifier/checkpoint-299/pytorch_model.bin (deflated 7%)
  adding: bengali_classifier/checkpoint-299/traine

## Arabic

In [ ]:
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")

enforce_reproducibility()

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenized_tydiqa = tydiqa['arabic'].map(preprocess_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="arabic_classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps = 100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_tydiqa["train"],
    eval_dataset=tokenized_tydiqa["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("arabic_classifier_final")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.231300,0.238447,0.915878
2,0.176200,0.229928,0.932177
